In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/generated-results/generated_LexRank_sumy.csv
/kaggle/input/generated-results/generated_all_bert_ext_sum_simple.csv
/kaggle/input/generated-results/generated_spacy_topicrank.csv
/kaggle/input/generated-results/generated_spacy_textrank.csv
/kaggle/input/generated-results/generated_coref_sbert_ext_sum_simple.csv
/kaggle/input/generated-results/generated_all_Lsa_sumy.csv
/kaggle/input/generated-results/generated_coref_spacy_textrank.csv
/kaggle/input/generated-results/generated_coref_spacy_topicrank.csv
/kaggle/input/generated-results/generated_all_sbert_ext_sum_simple.csv
/kaggle/input/generated-results/generated_bert_ext_sum_simple.csv
/kaggle/input/generated-results/generated_all_spacy_textrank.csv
/kaggle/input/generated-results/generated_all_LexRank_sumy.csv
/kaggle/input/generated-results/generated_coref_seq_to_seq_bart-base.csv
/kaggle/input/generated-results/generated_coref_KLSummarizer_sumy.csv
/kaggle/input/generated-results/generated_coref_Lsa_sumy.csv
/kaggle/inpu

In [ ]:
pip install sumy

In [2]:
val_df = pd.read_csv('/kaggle/input/tmp-results/generated_coref_validation.csv', sep=',')

In [6]:
sample = val_df.sample()
sample['clean_text'].iloc[0]

" . remember how you used to say . you could not wait 'til tomorrow for a brand new day? . and no fuss when you had to ride the bus . you would just add a little blush to paralyze your school crush . now you are older and the weight is on your shoulder . makes the world a little colder, no more hidin' in the old days . be strong, do not you give up hope . it will get hard, life like a jump rope . up, down, up, down . up, down, up, down, yeah . because it will get hard . remember, life like a jump rope . up, down, up, down . up, down, up, down, yeah . it will get hard . because it will get hard . there'll be a bump and there will be a bruise . there'll be alarms and there will be a snooze . there'll be a path that you will have to choose . there'll be a win and there will be a lose . and you gotta to hold your head up high and . watch all the negative go by . do not ever be ashamed to cry, you go ahead . because life like a jump rope . up, down, up, down . up, down, up, down, yeah . it 

In [21]:
sample['clean_summary'].iloc[0]

'because it will get hard .  remember lifes like a jump rope .  come on .  i want to tell you that everything will be okay'

In [9]:
text = sample['clean_text'].iloc[0]

In [15]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.luhn import LuhnSummarizer

In [17]:
sent_count = 4

In [25]:
parser = PlaintextParser.from_string(text,Tokenizer("english"))
summarizer_luhn = LuhnSummarizer()
sentences = summarizer_luhn(parser.document, sent_count)
result = ' '
for sent in sentences:
    result += (sent._text + ' ')

In [29]:
for sentence in summarizer_luhn(parser.document, sent_count):
    print(sentence)

it will get hard, life like a jump rope .
you stomp your feet so hard you make it pound .
raise it back up to the top, and now we are never coming down .
raise it back up to the top, and now we are never coming down .


In [47]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals


from collections import namedtuple
from operator import attrgetter
from sumy.utils import ItemsCount
from sumy._compat import to_unicode
from sumy.nlp.stemmers import null_stemmer


SentenceInfo = namedtuple("SentenceInfo", ("sentence", "order", "rating",))


class AbstractSummarizer(object):
    def __init__(self, stemmer=null_stemmer):
        if not callable(stemmer):
            raise ValueError("Stemmer has to be a callable object")

        self._stemmer = stemmer

    def __call__(self, document, sentences_count):
        raise NotImplementedError("This method should be overriden in subclass")

    def stem_word(self, word):
        return self._stemmer(self.normalize_word(word))

    @staticmethod
    def normalize_word(word):
        return to_unicode(word).lower()

    @staticmethod
    def _get_best_sentences(sentences, count, rating, *args, **kwargs):
        rate = rating
        if isinstance(rating, dict):
            assert not args and not kwargs
            def rate(s): return rating[s]

        infos = (SentenceInfo(s, o, rate(s, *args, **kwargs))
            for o, s in enumerate(sentences))

        # sort sentences by rating in descending order
        infos = sorted(infos, key=attrgetter("rating"), reverse=True)
        
        print(infos[0])
        
        # get `count` first best rated sentences
        if not callable(count):
            count = ItemsCount(count)
        infos = count(infos)
        # sort sentences by their order in document
        infos = sorted(infos, key=attrgetter("order"))

        return tuple(i.sentence for i in infos)

In [48]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.models import TfDocumentModel
# from sumy.summarizers._summarizer import AbstractSummarizer

class LuhnSummarizer(AbstractSummarizer):
    max_gap_size = 4
    # TODO: better recognition of significant words (automatic)
    significant_percentage = 1
    _stop_words = frozenset()

    @property
    def stop_words(self):
        return self._stop_words

    @stop_words.setter
    def stop_words(self, words):
        self._stop_words = frozenset(map(self.normalize_word, words))

    def __call__(self, document, sentences_count):
        words = self._get_significant_words(document.words)
        return self._get_best_sentences(document.sentences,
            sentences_count, self.rate_sentence, words)

    def _get_significant_words(self, words):
        words = map(self.normalize_word, words)
        words = tuple(self.stem_word(w) for w in words if w not in self._stop_words)

        model = TfDocumentModel(words)

        # take only best `significant_percentage` % words
        best_words_count = int(len(words) * self.significant_percentage)
        words = model.most_frequent_terms(best_words_count)

        # take only words contained multiple times in document
        return tuple(t for t in words if model.term_frequency(t) > 1)

    def rate_sentence(self, sentence, significant_stems):
        ratings = self._get_chunk_ratings(sentence, significant_stems)
        return max(ratings) if ratings else 0

    def _get_chunk_ratings(self, sentence, significant_stems):
        chunks = []
        NONSIGNIFICANT_CHUNK = [0]*self.max_gap_size

        in_chunk = False
        for order, word in enumerate(sentence.words):
            stem = self.stem_word(word)
            # new chunk
            if stem in significant_stems and not in_chunk:
                in_chunk = True
                chunks.append([1])
            # append word to chunk
            elif in_chunk:
                is_significant_word = int(stem in significant_stems)
                chunks[-1].append(is_significant_word)

            # end of chunk
            if chunks and chunks[-1][-self.max_gap_size:] == NONSIGNIFICANT_CHUNK:
                in_chunk = False

        return tuple(map(self._get_chunk_rating, chunks))

    def _get_chunk_rating(self, chunk):
        chunk = self.__remove_trailing_zeros(chunk)
        words_count = len(chunk)
        assert words_count > 0

        significant_words = sum(chunk)
        if significant_words == 1:
            return 0
        else:
            return significant_words**2 / words_count

    def __remove_trailing_zeros(self, collection):
        """Removes trailing zeroes from indexable collection of numbers"""
        index = len(collection) - 1
        while index >= 0 and collection[index] == 0:
            index -= 1

        return collection[:index + 1]

In [52]:
parser = PlaintextParser.from_string(text,Tokenizer("english"))
summarizer_luhn = LuhnSummarizer()
sentences = summarizer_luhn(parser.document, sent_count)
result = ' '
for sent in sentences:
    result += (sent._text + ' ')
result

SentenceInfo(sentence=<Sentence: raise it back up to the top, and now we are never coming down .>, order=42, rating=14.0)


' it will get hard, life like a jump rope . you stomp your feet so hard you make it pound . raise it back up to the top, and now we are never coming down . raise it back up to the top, and now we are never coming down . '

In [26]:
result

' it will get hard, life like a jump rope . you stomp your feet so hard you make it pound . raise it back up to the top, and now we are never coming down . raise it back up to the top, and now we are never coming down . '

In [55]:
import spacy

nlp = spacy.load("en_core_web_sm")

text = sample['clean_text'].iloc[0]

doc = nlp(text)
    
for token in doc:
    if token.pos_ == 'VERB':
        print(token.text, token.pos_)

remember VERB
used VERB
say VERB
wait VERB
had VERB
ride VERB
add VERB
paralyze VERB
makes VERB
give VERB
get VERB
get VERB
remember VERB
get VERB
get VERB
choose VERB
got VERB
hold VERB
watch VERB
go VERB
cry VERB
go VERB
get VERB
remember VERB
get VERB
c'm VERB
want VERB
tell VERB
turn VERB
keep VERB
pushing VERB
follow VERB
lead VERB
lose VERB
stomp VERB
make VERB
pound VERB
raise VERB
coming VERB
spin VERB
clap VERB
slip VERB
make VERB
pound VERB
raise VERB
coming VERB
get VERB
remember VERB
get VERB
get VERB


In [ ]:
summarizer_luhn = LuhnSummarizer()

def generate_summary(text):
    parser = PlaintextParser.from_string(text,Tokenizer("english"))
    sentences = summarizer_luhn(parser.document, sent_count)
    result = ' '
    for sent in sentences:
        result += (sent._text + ' ')
    result

In [ ]:
val_df = pd.read_csv('/kaggle/input/tmp-results/generated_coref_validation.csv', sep=',')

temp_df = val_df.sample(10)

In [ ]:
start = time.time()

temp_df['generated_summary'] = temp_df['clean_text'].apply(lambda x:generate_summary(x))
temp_df['coref_generated_summary'] = temp_df['coref_text'].apply(lambda x:generate_summary(x))

end = time.time()
print('time: ', end - start)